## Parametric Insurance Risk Analysis Interface
This notebook follows an actuarial workflow, proceeding from data setup to policy definition, trigger calibration, and final risk pricing. It uses the robust Percentile Trigger method to ensure triggers are set correctly for highly skewed rainfall data.

### Data and Environment Setup

This block installs dependencies, imports the core analysis functions, and sets up the necessary directories. It also checks for and downloads the 30 years of IMD historical data required for climatology calculations.

In [1]:
# Install necessary libraries (Only needed if running for the first time)
#!pip install imdlib pandas numpy matplotlib
%load_ext autoreload
%autoreload 2
import os
# Import all analysis functions from the core engine module
from parametric_core import (
    get_climatological_data,
    run_live_parametric_engine,
    calculate_historical_loss_expectancy,
    create_payoff_frequency_heatmap,
    analyze_aep_and_distribution
)

# Import data handling functions from the new module
from data_ingestion import ensure_historical_data_exists, setup_data_directories


### Define Policy and Model Parameters

Define all geographical, contractual, and modeling parameters here. This is the only place you need to change inputs.

In [ ]:
# =============================================================================
# DEFINE CONTRACTUAL AND GEOGRAPHICAL PARAMETERS
# =============================================================================

# --- Geographical Target ---
TARGET_LAT = 19.0760  # Example: Mumbai, Maharashtra
TARGET_LON = 72.8777

# --- Contractual Limits ---
FULL_PAYOUT_AMOUNT = 10000.0  # Maximum payout amount (e.g., in INR or USD)
POLICY_START_DATE = '2025-06-01'
POLICY_END_DATE = '2025-09-30' 

# --- Data Directories ---
DATA_DIR = 'IMD_Rain_Data'
LPA_FILE = 'IMD_Rain_Data/LPA/climatology_data.csv' 

# --- Index Definition ---
CUMULATIVE_DAYS = 1  # For a single-day extreme rainfall analysis
LPA_START_YR = 1981
LPA_END_YR = 2010 

# --- Fixed Full Payout Threshold ---
# This is a fixed severity trigger based on engineering risk (e.g., 150mm floods our site)
FINAL_HIGH_RAINFALL_THRESHOLD_MM = 204.5

print(f"Policy defined for {TARGET_LAT}°N, {TARGET_LON}°E. Max Payout: {FULL_PAYOUT_AMOUNT:,.2f}.")
print(f"Full Payout Trigger (Fixed): {FINAL_HIGH_RAINFALL_THRESHOLD_MM} mm.")

Policy defined for 19.076°N, 72.8777°E. Max Payout: 10,000.00.
Full Payout Trigger (Fixed): 204.5 mm.


### Calibrate the Partial Payout Trigger

This is the most critical step. We define the percentile rank (the rarity) we want the partial trigger to represent, and the function calculates the precise rainfall amount in millimeters corresponding to that percentile only for your location.

In [ ]:
# =============================================================================
# CALIBRATE PARTIAL PAYOUT TRIGGER (USING PERCENTILE)
# =============================================================================

# --- Risk Calibration Parameter (Adjust this value to change the policy's rarity) ---
# 99.9 means the trigger will only be hit by the top 0.1% of all historical rain days.
PARTIAL_TRIGGER_PERCENTILE = 99.9 

# Run the calculation to get the localized baseline and the percentile value.
lpa_daily, std_dev_daily, percentile_trigger_val = get_climatological_data(
    TARGET_LAT, TARGET_LON, DATA_DIR, LPA_FILE, LPA_START_YR, LPA_END_YR, PARTIAL_TRIGGER_PERCENTILE)

if percentile_trigger_val is not None:
    # Set the variable for the core engine based on the calculated percentile value
    PARTIAL_TRIGGER_VALUE = percentile_trigger_val
    
    # Safety check: ensure partial trigger is strictly below the full threshold
    if PARTIAL_TRIGGER_VALUE >= FINAL_HIGH_RAINFALL_THRESHOLD_MM:
         PARTIAL_TRIGGER_VALUE = FINAL_HIGH_RAINFALL_THRESHOLD_MM * 0.75
    
    print("\n--- Localized Climatology Results ---")
    print(f"LPA Daily Rainfall (Baseline): {lpa_daily:.2f} mm")
    print(f"Std Dev Daily Rainfall: {std_dev_daily:.2f} mm")
    print(f"🎯 Calculated Partial Payout Trigger ({PARTIAL_TRIGGER_PERCENTILE:.1f}% Rank): {PARTIAL_TRIGGER_VALUE:.2f} mm")
else:
    PARTIAL_TRIGGER_VALUE = 0.0
    print("❌ Critical: Percentile Trigger Calculation Failed.")


--- Calculating Climatological LPA and Percentile Trigger for Target Point (19.0760°N, 72.8777°E) ---
✅ Loaded LPA, Std Dev, and 99.9th Percentile from cache.

--- Localized Climatology Results ---
LPA Daily Rainfall (Baseline): 7.56 mm
Std Dev Daily Rainfall: 23.72 mm
🎯 Calculated Partial Payout Trigger (99.9% Rank): 153.38 mm


### Validation and Pricing

This block runs the full historical stress test, using your newly calibrated partial trigger and the fixed full trigger against $30 \text{ years}$ of data. The output provides the Expected Annual Loss (EAL), which forms the basis of the insurance premium.

In [ ]:
# 1. Run the Stress Test (financial risk metrics: EAL/MAL)
calculate_historical_loss_expectancy(
    TARGET_LAT, 
    TARGET_LON, 
    DATA_DIR, 
    LPA_FILE, 
    LPA_START_YR, 
    LPA_END_YR, 
    FINAL_HIGH_RAINFALL_THRESHOLD_MM,
    PARTIAL_TRIGGER_PERCENTILE, 
    CUMULATIVE_DAYS,
    FULL_PAYOUT_AMOUNT
)


### Trigger Validation - AEP and Return Period (RP)

This step takes the Calculated Partial Trigger Value from Step 3 and the Full Payout Threshold from Step 2 and validates them against the full historical Annual Maximum series. This analysis generates the Annual Exceedance Probability (AEP) and Return Period (RP) table, which is essential for risk comparison.

In [ ]:
# 1. AEP and Distribution Analysis 
# This shows the Annual Exceedance Probability and the Return Period 
# for the chosen PARTIAL_TRIGGER_PERCENTILE.
analyze_aep_and_distribution(
    TARGET_LAT, 
    TARGET_LON, 
    DATA_DIR, 
    LPA_START_YR, 
    LPA_END_YR,
    lpa_daily,             
    std_dev_daily,         
    PARTIAL_TRIGGER_VALUE, 
    FINAL_HIGH_RAINFALL_THRESHOLD_MM,
    CUMULATIVE_DAYS        
)



📊 AEP & DISTRIBUTION ANALYSIS (Single-Day Index) 📊
Period: 1990-2024


### Geographical Risk Assessment (Heatmap)

This step provides the spatial context for the risk. The heatmap shows the historical frequency (AEP) of the Partial Payout Trigger being hit across the entire geographical region, allowing you to compare the target location's risk profile against its neighbors.

In [ ]:
# 2. Generate the Risk Map (geographical pricing assessment)
create_payoff_frequency_heatmap(
    TARGET_LAT,
    TARGET_LON, 
    DATA_DIR, 
    LPA_START_YR, 
    LPA_END_YR,
    FINAL_HIGH_RAINFALL_THRESHOLD_MM,
    PARTIAL_TRIGGER_PERCENTILE, 
    CUMULATIVE_DAYS
)


### Final Live Check

This is the final application of the calibrated model, showing what the financial payout would be if the policy were active today, based on the latest available real-time index data.

In [ ]:
# 3. Run the live check (real-time payout decision)
run_live_parametric_engine(
    TARGET_LAT, 
    TARGET_LON, 
    DATA_DIR, 
    LPA_FILE, 
    LPA_START_YR, 
    LPA_END_YR, 
    FINAL_HIGH_RAINFALL_THRESHOLD_MM,
    PARTIAL_TRIGGER_PERCENTILE,
    CUMULATIVE_DAYS,
    FULL_PAYOUT_AMOUNT,
    POLICY_START_DATE,
    POLICY_END_DATE
)